In [1]:
# ============================================================================
# IMPORTS
# ============================================================================
import os
import numpy as np
import pandas as pd
from pathlib import Path
from astropy.io import fits
from astropy.time import Time

import gt_apps as my_apps

# NumPy compatibility
if not hasattr(np, "float"):
    np.float = float

print("Imports complete")

# ============================================================================
# TIME / TARGET HELPERS
# ============================================================================
FERMI_EPOCH = Time('2001-01-01T00:00:00', scale='utc')

def utc_to_met(t0, time_kind="auto"):
    """
    Return Fermi MET (float seconds) from either:
      - UTC string / astropy.Time  -> convert to MET (float)
      - numeric (int/float)        -> assume already MET (keep fractional)
    time_kind: 'auto' | 'utc' | 'met'
    """
    if time_kind == "met":
        return float(t0)
    if time_kind == "utc":
        t = t0 if isinstance(t0, Time) else Time(str(t0), scale='utc', format='iso')
        return (t - FERMI_EPOCH).to_value('sec')
    if isinstance(t0, (int, float)):
        return float(t0)
    if isinstance(t0, Time):
        return (t0 - FERMI_EPOCH).to_value('sec')
    return (Time(str(t0), scale='utc', format='iso') - FERMI_EPOCH).to_value('sec')

def build_target(name, ra_deg, dec_deg, t0, dt_s, time_kind="auto"):
    t0_met = float(utc_to_met(t0, time_kind=time_kind))
    dt_s = float(dt_s)
    return {
        "name": str(name),
        "ra": float(ra_deg),
        "dec": float(dec_deg),
        "t0_met": t0_met,
        "dt": dt_s,
        "tmin": t0_met,
        "tmax": t0_met + dt_s,
    }

# ============================================================================
# INPUT LIST & WINDOW
# ============================================================================
BASE = Path("/Users/salim/Desktop/MGFs_LAT/MGF_LAT_analysis_trigger")
MGF_LIST = BASE / "GBM_eMGF_candidates.txt"

# use 10 000 s window here
DELTA_T = 10000.0

df = pd.read_csv(MGF_LIST, sep=r"\s+", comment="#")

targets = [
    build_target(
        row.GRB_name,
        float(row.gal_ra_deg),
        float(row.gal_dec_deg),
        float(row.MET_trig_time),
        DELTA_T,
        time_kind="met",
    )
    for _, row in df.iterrows()
]

print(f"Loaded {len(targets)} MGF targets from {MGF_LIST}")
for t in targets:
    print(t)

# ============================================================================
# CONFIG FOR EVENT SELECTION
# ============================================================================
DATA_DIR = BASE
WORK_DIR = BASE / "MGF_Exposure_Check"
WORK_DIR.mkdir(parents=True, exist_ok=True)

EMIN = 100.0       # MeV
EMAX = 10000.0     # MeV
ROI  = 12.0        # deg
ZMIN = 0.0
ZMAX = 100.0
EVCLASS = 8        # Transient
EVTYPE  = 3        # FRONT+BACK

# ============================================================================
# FILE DISCOVERY
# ============================================================================
def find_ft1_ft2(data_dir: Path):
    """Return FT1 (events) and FT2 (spacecraft) from a GRB_* folder."""
    ft1_patterns = [
        "L*_EV00.fits", "*_EV00.fits", "*_EV*.fits",
        "L*_PH00.fits", "*_PH00.fits", "*_PH*.fits",
        "gll_ft1_tr_*.fit", "gll_ft1_*.fit",
    ]
    ft2_patterns = [
        "L*_SC00.fits", "*_SC00.fits", "*_SC*.fits",
        "spacecraft*.fits", "*_SC.fits",
    ]
    def first_match(patterns):
        for pat in patterns:
            files = sorted(data_dir.glob(pat))
            if files:
                return str(files[0])
        return None

    ft1 = first_match(ft1_patterns)
    ft2 = first_match(ft2_patterns)
    if not ft1 or not ft2:
        raise FileNotFoundError(f"Missing FT1/FT2 in {data_dir}. Found FT1={ft1}, FT2={ft2}")
    return ft1, ft2

# ============================================================================
# GTSELECT / GTMKTIME WRAPPERS
# ============================================================================
def gtselect_filter_events(ifl, gtselect_outfile, evclass, evtype, ra, dec, rad,
                           tmin, tmax, emin, emax, zmin, zmax):
    my_apps.filter['infile']  = str(ifl)
    my_apps.filter['outfile'] = str(gtselect_outfile)
    my_apps.filter['evclass'] = int(evclass)
    my_apps.filter['evtype']  = int(evtype)
    my_apps.filter['ra']      = float(ra)
    my_apps.filter['dec']     = float(dec)
    my_apps.filter['rad']     = float(rad)
    my_apps.filter['tmin']    = float(tmin)
    my_apps.filter['tmax']    = float(tmax)
    my_apps.filter['emin']    = float(emin)
    my_apps.filter['emax']    = float(emax)
    my_apps.filter['zmin']    = float(zmin)
    my_apps.filter['zmax']    = float(zmax)
    print("[gtselect]", my_apps.filter.command())
    my_apps.filter.run()
    return gtselect_outfile

def gtmaketime_events(scfile, evfile, gtmaketime_ofl, ra_src, dec_src, roi_deg, zmax_deg):
    # limb cut via GTIs: ANGSEP <= (zmax - ROI)
    filt = (
        "(DATA_QUAL>0)&&(LAT_CONFIG==1)&&(IN_SAA==F)&&(LIVETIME>0)&&"
        f"(ANGSEP(RA_ZENITH,DEC_ZENITH,{ra_src:.6f},{dec_src:.6f})<={zmax_deg - roi_deg:.1f})"
    )
    my_apps.maketime['scfile']   = str(scfile)
    my_apps.maketime['sctable']  = "SC_DATA"
    my_apps.maketime['filter']   = filt
    my_apps.maketime['roicut']   = "no"
    my_apps.maketime['evfile']   = str(evfile)
    my_apps.maketime['evtable']  = "EVENTS"
    my_apps.maketime['outfile']  = str(gtmaketime_ofl)
    print("[gtmktime]", my_apps.maketime.command())
    my_apps.maketime.run()
    return gtmaketime_ofl

# ============================================================================
# MAIN LOOP: FIND FIRST EXPOSURE TIME
# ============================================================================
for target in targets:
    name = target["name"]
    print("\n" + "="*60)
    print(f"Checking exposure for {name} (ΔT = {DELTA_T} s)")
    print("="*60)

    data_dir = DATA_DIR / name
    work_dir = WORK_DIR / name
    work_dir.mkdir(parents=True, exist_ok=True)

    try:
        ft1, ft2 = find_ft1_ft2(data_dir)
    except Exception as e:
        print(f"  Skip {name}: missing FT1/FT2 ({e})")
        continue

    filtered = work_dir / "filtered_10ks.fits"
    gti_file = work_dir / "gti_10ks.fits"

    # 1) gtselect over 10 ks
    gtselect_filter_events(
        ifl=ft1,
        gtselect_outfile=filtered,
        evclass=EVCLASS, evtype=EVTYPE,
        ra=target["ra"], dec=target["dec"], rad=ROI,
        tmin=target["tmin"], tmax=target["tmax"],
        emin=EMIN, emax=EMAX,
        zmin=ZMIN, zmax=ZMAX
    )

    # quick event count
    with fits.open(filtered) as hdul:
        n_events = len(hdul["EVENTS"].data) if "EVENTS" in hdul else 0
    print(f"  Events after gtselect: {n_events}")

    if n_events == 0:
        print("  No events in 10 ks window -> no exposure info from GTIs (for this ROI).")
        continue

    # 2) gtmktime to build GTIs including limb cut
    gtmaketime_events(
        scfile=ft2, evfile=filtered, gtmaketime_ofl=gti_file,
        ra_src=target["ra"], dec_src=target["dec"],
        roi_deg=ROI, zmax_deg=ZMAX
    )

    # 3) read GTI extension and find first exposure start
    with fits.open(gti_file) as hdul:
        if "GTI" not in hdul or len(hdul["GTI"].data) == 0:
            print("  GTI table is empty -> effectively no good exposure in this 10 ks window.")
            continue

        gtab = hdul["GTI"].data
        starts = gtab["START"]
        stops  = gtab["STOP"]

        t_start_first = float(np.min(starts))
        t_stop_last   = float(np.max(stops))

    dt_start = t_start_first - target["t0_met"]
    dt_stop  = t_stop_last  - target["t0_met"]

    print(f"  First GTI START: {t_start_first:.3f} (MET)")
    print(f"  Last  GTI STOP : {t_stop_last:.3f} (MET)")
    print(f"  -> First exposure at t - t_trig ≈ {dt_start:.1f} s")
    print(f"  -> Last  exposure at t - t_trig ≈ {dt_stop:.1f} s")


/var/folders/st/ng23bqyd59ndbk5_qmpyq8rr0000gn/T/ipykernel_17026/4271576155.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if not hasattr(np, "float"):


Imports complete
Loaded 7 MGF targets from /Users/salim/Desktop/MGFs_LAT/MGF_LAT_analysis_trigger/GBM_eMGF_candidates.txt
{'name': 'GRB_231115A', 'ra': 148.968458, 'dec': 69.679703, 't0_met': 721755386.0, 'dt': 10000.0, 'tmin': 721755386.0, 'tmax': 721765386.0}
{'name': 'GRB_200415A', 'ra': 11.888058, 'dec': -25.2888, 't0_met': 608633290.0, 'dt': 10000.0, 'tmin': 608633290.0, 'tmax': 608643290.0}
{'name': 'GRB_180128A', 'ra': 11.888058, 'dec': -25.2888, 't0_met': 538809001.0, 'dt': 10000.0, 'tmin': 538809001.0, 'tmax': 538819001.0}
{'name': 'GRB_120616A', 'ra': 56.702142, 'dec': 68.096106, 't0_met': 361552012.0, 'dt': 10000.0, 'tmin': 361552012.0, 'tmax': 361562012.0}
{'name': 'GRB_200423A', 'ra': 308.71805, 'dec': 60.153678, 't0_met': 609342856.0, 'dt': 10000.0, 'tmin': 609342856.0, 'tmax': 609352856.0}
{'name': 'GRB_231024A', 'ra': 11.888058, 'dec': -25.2888, 't0_met': 719846439.0, 'dt': 10000.0, 'tmin': 719846439.0, 'tmax': 719856439.0}
{'name': 'GRB_081213A', 'ra': 11.888058, 'dec'